In [1]:
import os
import torch
import numpy as np

def trans(img):
    return np.array(np.transpose(img, [1, 2, 0]), dtype=np.int32)

base_path = '../checkpoint/valid/kaggle.pt'
masks_path = os.path.join(base_path, 'masks.npy')
ids_path = os.path.join(base_path, 'ids')

masks_pr = np.load(masks_path)
with open(ids_path, 'r') as fin:
    ids = np.array(fin.read().strip().split('\n'))
len(ids), masks_pr.shape

(555, (555, 4, 320, 640))

In [2]:
import sys
sys.path.append('..')

from utils import *
from dataset import Readdata, Cloudset

def debug_augmentation():
    ar = [
        albu.Resize(350, 525)
    ]
    return albu.Compose(ar)

path ='../../data'
r =  Readdata(path)
valid_set = Cloudset(
    r.train,
    'all',
    ids,
    r.train_fold,
    debug_augmentation()
)
valid_set.ids[:10]

array(['fb2421c.jpg', '29dfbc9.jpg', '3b75975.jpg', 'b24b4f6.jpg',
       '280fb45.jpg', 'bf287e6.jpg', '5f944a6.jpg', '025ad96.jpg',
       'f7fcf8c.jpg', '0a7a247.jpg'], dtype='<U11')

In [6]:
def draw_convex_hull(mask, mode='convex'):
    
    img = np.zeros(mask.shape)
    contours, hier = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        if mode=='rect': # simple rectangle
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), -1)
        if mode=='convex': # minimum convex hull
            hull = cv2.convexHull(c)
            cv2.drawContours(img, [hull], 0, (255, 255, 255),-1)
        else: # minimum area rectangle
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(img, [box], 0, (255, 255, 255),-1)
    return img/255

def dice(x, y, eps=1e-7):
    return (np.sum(x * y) + eps) / (np.sum(x) + np.sum(y) + eps)

def post_process(probability, threshold, min_size):
    """
    This is slightly different from other kernels as we draw convex hull here itself.
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    # don't remember where I saw it
    mask = (cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1])
    mask = draw_convex_hull(mask.astype(np.uint8))
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((350, 525), np.float32)
    num = 0
    for c in range(1, num_component):
        p = component == c
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

n = len(valid_set.ids)
gt_m = []
for i in range(n):
    gt_m.append(valid_set.__getitem__(i)[1])
    if i % 50 == 0:
        print('%s/%s'  % ((i+1), n))
gt_m = np.asarray(gt_m, dtype=np.float32)
print(gt_m.shape)

1/555
51/555
101/555
151/555
201/555
251/555
301/555
351/555
401/555
451/555
501/555
551/555
(555, 4, 350, 525)
debug 0:


NameError: name 'draw_convex_hull' is not defined

In [12]:
for j in range(4):
    print('debug %s:' % j)
    res = []
    for t in range(0, 100, 5):
        thr = t / 100
        for ms in [0, 100, 1200, 5000, 10000, 30000]:
            d = 0
            for i in range(n):
                predict, predict_num = post_process(resize_f(masks_pr[i, j]), thr, ms)
                d += dice(predict, gt_m[i, j])
            res.append((t, ms, d / n))
            print('%s\t%s\t%s'%(t, ms, d/n))

debug 0:
0	0	0.10591485274610674
0	100	0.10591485274610674
0	1200	0.10591485274610674
0	5000	0.10591485274610674
0	10000	0.10591485274610674
0	30000	0.10591485274610674
5	0	0.13379812892094842
5	100	0.13392883531681882
5	1200	0.13782094778106221
5	5000	0.14375333994010542
5	10000	0.14632128920847443
5	30000	0.20934551543936258
10	0	0.1451640947266287
10	100	0.14701424931419887
10	1200	0.15589534820736495
10	5000	0.1830999124256184
10	10000	0.2041956803926341
10	30000	0.3663218026279472
15	0	0.1495595608825422
15	100	0.15496958351247977
15	1200	0.1765403260027325
15	5000	0.21650060054311937
15	10000	0.2517438280762381
15	30000	0.42787377269484933
20	0	0.1524826051738745
20	100	0.16687204490343083
20	1200	0.1975517718706367
20	5000	0.2560294478921436
20	10000	0.30987798295080266
20	30000	0.4741097097962487
25	0	0.16343450755490121
25	100	0.17422397565846
25	1200	0.22435675470935096
25	5000	0.28435932464964137
25	10000	0.3425343274631677
25	30000	0.5027234962891606
30	0	0.1680558148896206